In [1]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#retrieve and format data - into labels from the dataset
def labels(filename):
    data = pd.read_csv(filename)
    data_y = data['LABEL']
    data_y-= 1
    return data_y.transpose()

In [3]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [4]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [5]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
        training_data_y = np.load(f)
        test_y = np.load(f)

training_data_x = training_data_x.transpose()
training_data_y = training_data_y.transpose()
training_data_y = training_data_y.reshape(1,training_data_x.shape[1])

#set hyperparameters & variables
learning_rate = 0.00003
epochs = 300
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

In [6]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [7]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })

    if t % display_step == 0:
        print("Epoch = %g, Loss = %g, Weights = %s, Bias = %s" % (t, current_loss, str(current_W), str(current_b)))
    
    loss_arr.append(current_loss)
    #acc_arr.append(accuracy(logistic_layer(current_W * training_data_x + current_b), training_data_y))
    acc_arr.append(accuracy(logistic_layer(np.dot(current_W, training_data_x) + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "Weights=", session.run(W), "Bias=", session.run(b), '\n')

2021-12-15 10:01:34.931351: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 10:01:34.940888: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 10:01:34.941443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-15 10:01:34.942431: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch = 0, Loss = 0.25, Weights = [[0. 0. 0. ... 0. 0. 0.]], Bias = [0.]
Epoch = 5, Loss = 0.175745, Weights = [[-6.8771674e-06 -8.4137773e-06  2.2227656e-05 ...  7.9368248e-05
   1.0074141e-04  7.4370815e-05]], Bias = [6.884479e-05]
Epoch = 10, Loss = 0.135767, Weights = [[-1.24890075e-05 -1.51615141e-05  3.78293407e-05 ...  1.40106975e-04
   1.80216608e-04  1.29618071e-04]], Bias = [0.00012523]
Epoch = 15, Loss = 0.111991, Weights = [[-1.6678932e-05 -2.0119169e-05  4.9239949e-05 ...  1.8811712e-04
   2.4414764e-04  1.7104899e-04]], Bias = [0.00017256]
Epoch = 20, Loss = 0.0970477, Weights = [[-1.9799907e-05 -2.3744555e-05  5.7642119e-05 ...  2.2679895e-04
   2.9655205e-04  2.0230786e-04]], Bias = [0.00021304]
Epoch = 25, Loss = 0.0871523, Weights = [[-2.2181714e-05 -2.6460939e-05  6.3784733e-05 ...  2.5849597e-04
   3.4034575e-04  2.2600745e-04]], Bias = [0.00024824]
Epoch = 30, Loss = 0.0802627, Weights = [[-2.4063766e-05 -2.8578343e-05  6.8200228e-05 ...  2.8491506e-04
   3.7766239

In [8]:
test_x = test_x.transpose()
test_y = test_y.transpose()
test_y = test_y.reshape(1,test_x.shape[1])

predicted_y = np.dot(current_W, test_x) + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  65.66371681415929 %


In [9]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
